In [2]:
import pymysql as db
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

DB 연결 및 SQL문 확인

In [2]:
# user = 'jyji'
# password = '123123'
# host = 'localhost'
# dbname = 'IMDB'
# charset = 'utf8'
con = db.connect(host='localhost', user='jyji', password='123123', db='IMDB', charset='utf8')
cur = con.cursor()
sql = 'select * from actors where id=20000;'
cur.execute(sql)
rows = cur.fetchall()
print(rows)
con.close()

((20000, 'Eric', 'Aude', 'M'),)


In [3]:
def get_connection():
    con = db.connect(host='localhost', user='jyji', password='123123', db='IMDB', charset='utf8')
    return con

In [4]:
with get_connection() as con:
    SQL = open('./SQL/movie_related_information.sql').read()
    df = pd.read_sql(SQL, con)
    print(df.head())
    df.to_csv('./data/movie_related_information.csv')

   MID                  Actor Movie              Role  Year
0    2  Darrell (I) Armstrong     $  Associated Press  1971
1    2          Warren Beatty     $       Joe Collins  1971
2    2        Scott (I) Brady     $             Sarge  1971
3    2          Arthur Brauss     $         Candy Man  1971
4    2             Gert Fröbe     $        Mr. Kessel  1971


In [5]:
with get_connection() as con:
    SQL = open('./SQL/director_related_information.sql').read()
    df = pd.read_sql(SQL, con)
    print(df.head())
    df.to_csv('./data/director_related_information.csv')

     MID        Director                           Movie  Year
0   1114  A. Aleksandrov                     14/1 endlos  1998
1  17209        A. Cyran                   Animali pazzi  1939
2  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975
3  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975
4  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975


Pandas Dataframe으로 변환

In [10]:
movie_df = pd.read_csv('./data/movie_related_information.csv', index_col=[0])
director_df = pd.read_csv('./data/director_related_information.csv', index_col=[0])

In [11]:
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell (I) Armstrong,$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott (I) Brady,$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


1세 2세 (I) (II) 와 같은 regnal number 뒤로 위치 바꾸기

In [12]:
def change_regnal_number(x):
    l = x.split(' ')
    if l[1].startswith('('):
        l[1], l[2] = l[2], l[1]
    return ' '.join(l)


movie_df['Actor'] = movie_df['Actor'].apply(change_regnal_number)
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell Armstrong (I),$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott Brady (I),$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


In [14]:
director_df.sample(10)

,MID,Director,Movie,Year
17181,4117,Roy Ward Baker,Aaghaaz,2000
16765,1529,Robert (I) Altman,2 Fast 2 Furious,2003
20345,3113,William Asher,8 Women,1999
20243,37249,Werner Jacobs,"Big Operator, The",1959
7476,15231,Harley Cokeliss,Amy's Orgasm,2001
4160,82364,David Veloz,Delightfully Dangerous,1945
20713,42209,Yeung Kong,Bljackor,1964
19745,31058,Val Guest,Battle Circus,1953
1245,74827,Anna Solanas,Cuentos erticos,1980
14596,73772,Ola Simonsson,"Cross of Lorraine, The",1943


In [18]:
director_df['Director'] = director_df['Director'].apply(change_regnal_number)
director_df.sample(10)

,MID,Director,Movie,Year
11231,66942,Kevin Rixon,Cold Heaven,1991
6432,87188,Gene Yates,Direktan prenos,1982
14454,73258,Nina Shorina (II),Crimebroker,1993
7286,26119,Guy Freeman (I),Babes in Arms,1939
5056,20235,Edward Dillon,Arizona,1940
2722,34670,Carl K. Hittleman,Best Laid Plans,1999
19298,8478,Tom Booker,Agostino,1962
6281,77643,Gábor Szöllõskei,"Dance of Life, The",1929
18614,83024,Suresh Vinu,Deprem,1976
9284,30736,Jerry Gross,Bastard!!,1992


In [26]:
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell Armstrong (I),$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott Brady (I),$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


두 테이블을 조인해서 다시 저장

In [32]:
joined_df = pd.merge(left=movie_df, right=director_df, how='inner', on=['MID', 'Movie', 'Year'])
joined_df = joined_df.drop('MID', axis=1)
joined_df = joined_df[['Actor', 'Director', 'Movie', 'Role', 'Year']]
joined_df.head(10)

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971
5,Bob Herron,Les 12 Poissons,$,Bodyguard,1971
6,Horst Hesslein,Les 12 Poissons,$,Bruno,1971
7,Hans Hutter (I),Les 12 Poissons,$,Karl,1971
8,Wolfgang Kieling,Les 12 Poissons,$,Granich,1971
9,Wolfgang Kuhlman,Les 12 Poissons,$,Furcoat,1971


In [33]:
movie_df.to_csv('./data/movie_related_information.csv')
director_df.to_csv('./data/director_related_information.csv')
joined_df.to_csv('./data/joined_table.csv')

In [3]:
movie_df = pd.read_csv('./data/movie_related_information.csv', index_col=[0])
director_df = pd.read_csv('./data/director_related_information.csv', index_col=[0])
joined_df = pd.read_csv('./data/joined_table.csv', index_col=[0])

단어 토큰 준비

In [4]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

for col in joined_df.columns:
    word_list = joined_df[col].tolist()
    word_list = list(map(str, word_list))
    word_list = list(set(word_list))
    len_word_dict = len(word_dict)
    for i, w in enumerate(word_list):
        word_dict[w] = i + len_word_dict

number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(number_dict)

len(word_dict), len(number_dict), vocab_size

(202880, 202880, 202880)

In [5]:
import random

random.sample(list(word_dict), 10), random.sample(list(number_dict), 10)

(['Dawn Lorraine',
  'Col. Mitchell McDonald',
  'Anna Blank',
  'Atte Hovimaa',
  'Robert Nash',
  'Platoon Sgt. William Maccone',
  'Minnie Summers Lindsey',
  'Nora Doel',
  'Iwataro Ishii',
  'Helmut Bein'],
 [52736, 2897, 59458, 54682, 167236, 41110, 196816, 73479, 125805, 61968])

joined dataframe을 복사하여 토큰 테이블로 바꾸기

In [6]:
token_df = joined_df.copy()

In [7]:
token_df.head()

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971


In [8]:
for col in token_df.columns:
    token_df[col] = token_df[col].apply(lambda x: str(x))

In [9]:
token_df.head()

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971


In [10]:
type(token_df['Year'][0])

str

In [11]:
def change_to_token(entity):
    return word_dict[entity]

for col in token_df.columns:
    token_df[col] = token_df[col].apply(change_to_token)

token_df.head()

,Actor,Director,Movie,Role,Year
0,85977,103573,108593,136876,202847
1,87969,103573,108593,188469,202847
2,81064,103573,108593,182880,202847
3,6986,103573,108593,187791,202847
4,23105,103573,108593,136541,202847


In [14]:
for entity in token_df.iloc[0, :].values:
    print(number_dict[entity], end=', ')

Darrell Armstrong (I), Richard Kwietniowski, $, The single man, 1977, 

토큰 테이블 저장

In [15]:
token_df.to_csv('./data/token_table.csv')